# 0 - Environment Set Up 

In [7]:
# Install required packages
!pip install -r ../requirements.txt

# 1 - Project Set Up

## 1.1 - Map to Root

In [8]:
REL_PATH_TO_ROOT = "../"

In [9]:
import sys
import os

In [10]:
sys.path.insert(0,REL_PATH_TO_ROOT)

In [11]:
from src.utils import get_root_dir, test_root_dir
from local_variables import ROOT_DIR

In [12]:
test_root_dir(REL_PATH_TO_ROOT)

Root set up correctly


## 1.2 - Set-Up Imports

In [13]:
from prompt_manager.manager import PromptManager
from prompt_manager.fetcher import fetch_prompt
from src.api import generate_outputs_openai

ModuleNotFoundError: No module named 'openai'

In [14]:
import pandas as pd

## 1.3 - Fictious Hallucination Dataset

In [15]:
# List of contexts for the chatbot
contexts = [
    "Provide information on the largest ocean on Earth.",
    "Explain the causes of the American Civil War.",
    "Give details about a historic battle that involved dragons.",
    "Summarize the work done by Nikola Tesla on wireless power transmission.",
    "Who won the FIFA World Cup in 2030?",
]

# List of chatbot responses to the contexts
responses = [
    "The Pacific Ocean is the largest ocean, covering more than 63 million square miles.",
    "The American Civil War was caused by multiple factors, including economic differences, states' rights, and the issue of slavery.",
    "The Battle of the Dragon Isles in the 14th century was a historic battle involving dragons and knights in medieval Europe.",
    "Nikola Tesla worked extensively on wireless power transmission, including his famous Wardenclyffe Tower project.",
    "The FIFA World Cup in 2030 was won by Argentina, defeating France in the finals with a score of 3-2."
]

# Ground truth list indicating if each response contains a hallucination
# True = contains a hallucination, False = accurate
is_hallucination_ground_truths = [
    False,  # Correct information about the Pacific Ocean
    False,  # Correct summary of American Civil War causes
    True,   # Hallucination: no historic battle with dragons is recorded
    False,  # Accurate summary of Tesla's work on wireless power
    True    # Hallucination: World Cup 2030 has not occurred yet
]

In [16]:
hal_df = pd.DataFrame({"context" : contexts, "chatbot_response" : responses, "hallucination_ground_truth" : is_hallucination_ground_truths})

In [17]:
hal_df.head()

,context,chatbot_response,hallucination_ground_truth
0,Provide information on the largest ocean on Ea...,"The Pacific Ocean is the largest ocean, coveri...",False
1,Explain the causes of the American Civil War.,The American Civil War was caused by multiple ...,False
2,Give details about a historic battle that invo...,The Battle of the Dragon Isles in the 14th cen...,True
3,Summarize the work done by Nikola Tesla on wir...,Nikola Tesla worked extensively on wireless po...,False
4,Who won the FIFA World Cup in 2030?,The FIFA World Cup in 2030 was won by Argentin...,True


# 2 - Run Your Evaluation 

## 2.1 - Get Your Prompts

In [18]:
SEQUENCE = ["task_1","hallucination_detector"]

prompt_template = fetch_prompt(SEQUENCE,use_latest_version=True)

print(prompt_template)

Here's the context:
    {CONTEXT}

    Here's what the AI said:

    {RESPONSE}

    Was the response a hallucination?
    
    Say 'True' if it's hallucination and 'False' if it's not a hallucination and nothing else!


In [19]:
evaluator_responses = []

# Loop through dataset
for _, row in hal_df.iterrows():

    # Get inputs and place into dictionary format
    context = row["context"]

    response = row["chatbot_response"]

    row_inputs = {"CONTEXT" : context, "RESPONSE" : response}

    # Initialise prompt to validate and format inputs
    prompt = PromptManager(template=prompt_template,inputs=row_inputs)
    prompt.validate_inputs()
    prompt.format_inputs()

    # Send prompt and collect response
    response = generate_outputs_openai(prompt.prompt)
    evaluator_responses.append(response)

hal_df["evaluator_response"] = evaluator_responses
display(hal_df.head(5))

Inputs validated successfully!
Previous validation successful, proceeding with formatting


NameError: name 'generate_outputs_openai' is not defined